In [2]:
import sys
sys.path.append('../src/scETM/')

In [3]:
import scanpy as sc
import numpy as np
import anndata as ad
import torch

/home/niujeffrey/anaconda3/envs/cpsc536r/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from batch_sampler import CellSamplerCITE
from models.multiETM import MultiETM
from trainers.UnsupervisedTrainerCITE import UnsupervisedTrainerCITE

In [5]:
rna_adata = ad.read_h5ad('./4_13176_raw_rna.h5ad')
protein_adata = ad.read_h5ad('./4_13176_raw_protein.h5ad')
rna_adata = ad.concat([rna_adata], label="batch_indices")
protein_adata = ad.concat([protein_adata], label="batch_indices")

In [6]:
model = MultiETM(protein_adata.n_vars,rna_adata.n_vars,  1)

n_batches == 1, disabling batch bias


In [7]:
trainer = UnsupervisedTrainerCITE(model,  protein_adata, rna_adata,test_ratio=0.1 )

In [ ]:
trainer.train(n_epochs=1000, eval_every=1000, eval_kwargs = dict(cell_type_col = 'cell_type'), n_samplers=1, save_model_ckpt=False)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [8]:
protein_adata.X

array([[ 0.,  0.,  5., ...,  2., 10., 10.],
       [ 0.,  5.,  6., ...,  4.,  6., 24.],
       [ 0.,  1., 13., ..., 35., 14.,  5.],
       ...,
       [ 1.,  0., 10., ...,  3., 28., 29.],
       [ 1.,  2.,  9., ...,  5.,  9., 50.],
       [ 0.,  3.,  4., ...,  3., 20., 43.]], dtype=float32)

In [8]:
a = torch.rand((200, 2000))
b = torch.rand((200, 140))

In [10]:
a = torch.Tensor([3, ])

torch.Size([200, 2140])

In [12]:
from torch.distributions import Independent, Normal
from torch.nn.functional import normalize

In [9]:
sampler = CellSamplerCITE(rna_adata, protein_adata, batch_size=10000, n_epochs = 1)
dataloader = iter(sampler)
data_dict = {k: v for k, v in next(dataloader).items()}

In [36]:
val = data_dict['cells_protein'] / data_dict['library_size_protein']
j = 0
for i in data_dict['library_size_protein'].detach().numpy():
    if i[0] == 0:
        print(i[0])
        print(j)
    j += 1

0.0
292
0.0
594
0.0
1068
0.0
2163
0.0
8178


In [33]:
layer = torch.nn.Linear(140, 128)

In [35]:
layer(val)

tensor([[ 0.0114, -0.0499, -0.0408,  ..., -0.0692, -0.0738,  0.0328],
        [ 0.0114, -0.0428, -0.0382,  ..., -0.0698, -0.0697,  0.0304],
        [ 0.0083, -0.0402, -0.0294,  ..., -0.0707, -0.0585,  0.0198],
        ...,
        [ 0.0065, -0.0590, -0.0401,  ..., -0.0604, -0.0843,  0.0443],
        [ 0.0113, -0.0485, -0.0424,  ..., -0.0693, -0.0644,  0.0315],
        [ 0.0105, -0.0490, -0.0387,  ..., -0.0712, -0.0716,  0.0326]],
       grad_fn=<AddmmBackward0>)

In [15]:
q = Independent(Normal(loc=a, scale=b), 1)
q.rsample()

tensor([[ 4.2549, -1.5641,  8.9214]])